In [71]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.multiclass import OneVsRestClassifier
import pandas as pd

try:
    from pygam import LogisticGAM
except ImportError as e:
    raise ImportError("pyGAM mangler. Kjør: pip install pygam") from e

import pandas as pd

In [72]:
df_my_data = pd.read_csv("./student-graduation/graduation_dataset_processed.csv")
df_head = df_my_data.head(4000)
#df = df_head.copy()  # eller bruk hele df_my_data i stedet for head(800)
df = df_my_data.copy()  # eller bruk hele df_my_data i stedet for head(800)
le = LabelEncoder()
df['Target'] = le.fit_transform(df['Target'])

In [ ]:
def get_model_and_params(model_type):
    model_type = str(model_type).strip().lower()

    if model_type == "random_forest":
        model = RandomForestClassifier(random_state=42, n_jobs=-1)
        param_grid = {
            'model__n_estimators': [100, 300],
            'model__max_depth': [None, 10, 20],
            'model__min_samples_split': [2, 5]
        }
    elif model_type == "svm":
        model = SVC(random_state=42)
        param_grid = {
            'model__C': [0.1, 1, 10],
            'model__kernel': ['linear', 'rbf'],
            'model__gamma': ['scale', 'auto']
        }
    elif model_type == "xgboost":
        model = XGBClassifier(eval_metric='mlogloss', random_state=42, n_jobs=-1)
        param_grid = {
            'model__n_estimators': [100, 300],
            'model__max_depth': [3, 6],
            'model__learning_rate': [0.01, 0.1]
        }
    elif model_type in ("neural_net", "mlp"):
        model = MLPClassifier(random_state=42, max_iter=1000, validation_fraction=0.20,n_iter_no_change=20,early_stopping=True, solver='adam',     )
        param_grid = {
            'model__hidden_layer_sizes': [(64,), (128,), (64, 64)],
            'model__alpha': [1e-4, 1e-3, 1e-2],
            'model__learning_rate_init': [1e-3, 1e-2],
            'model__activation': ['relu', 'tanh']
        }
    elif model_type == "additive":
         base = LogisticGAM(verbose=False)
         model = OneVsRestClassifier(base, n_jobs=None)  # la CV håndtere parallellisering
         param_grid = {
            # kraftig opp med lam for å hindre divergens
            "model__estimator__lam": [10, 100, 1000, 10000],
            # mindre kompleksitet per feature
            "model__estimator__n_splines": [5, 10, 20],
            # sørg for at vi har nok iterasjoner til å konvergere
            "model__estimator__max_iter": [20000],
            # litt romsligere toleranse kan hjelpe konvergens
            "model__estimator__tol": [1e-3],
    }
    else:
        raise ValueError("Ukjent modelltype")
    return model, param_grid

def run_pipeline(data: pd.DataFrame, target_column: str, model_type: str):
    if target_column not in data.columns:
        raise ValueError(f"Målkolonnen '{target_column}' finnes ikke i data.")

    X = data.drop(columns=[target_column])
    y = data[target_column]


    # Konservativ CV basert på minste klassefrekvens
    min_class_count = pd.Series(y).value_counts().min()
    cv = int(min(5, max(2, min_class_count)))

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    model, param_grid = get_model_and_params(model_type)

    pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='median')),  # trygghet hvis noen NaN finnes
        ('model', model)
    ])

    grid_search = GridSearchCV(
        pipeline,
        param_grid,
        cv=cv,
        scoring='accuracy',
        n_jobs=-1,
        refit=True,
        error_score='raise'
    )
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)

    print("✅ Beste parametere:", grid_search.best_params_)
    print("📊 Test accuracy:", acc)
    return best_model



In [74]:
try:
    _ = LogisticGAM  # referanse fra import-cellen
except NameError:
    raise ImportError("pyGAM ikke importert. Kjør import-cellen på nytt eller installer: python -m pip install pygam")

In [ ]:
# Mulige modeller å kjøre: "random_forest", "svm", "xgboost", "neural_net, "additive"
#Det må gjøres tuning på additive modellen for få noen resultater, så den er ikke ferdig enda, men de andre modelle fungerer.
model = run_pipeline(df, target_column='Target', model_type='svm')


OptimizationError: PIRLS optimization has diverged.
Try increasing regularization, or specifying an initial value for self.coef_